In [1]:
clean_up = True
%run stdPackages.ipynb
ws = gams.GamsWorkspace(working_directory=d['work']) # specify where you want to run the GAMS models from (here the repository referred to in d['work'])
d['data'] = os.path.join(d['data'], 'IO2018')

### 1. Settings

Define main settings:

In [2]:
name = 'GR18' # name of model
module = name+'_Production' # name of module
db_IO = GpyDB(pickle_path = os.path.join(d['data'], f'IO_{name}')) # load IO database named IO_name
with open(f"{d['data']}\\glob_{name}","rb") as file: # load global settings anmed glob_name
    glob=pickle.load(file)

#### 1.1 Nesting

**Non-materials nest:**

*Full index:*

In [3]:
mFull = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), 
                                   ('KELBM', 'KELB'), 
                                   ('KELB', 'iB'), ('KELB','KEL'), 
                                   ('KEL','L'), ('KEL','KE'), 
                                   ('KE','iM'), ('KE','E'), 
                                   ('E','35011_input'), ('E','35011_F')], names = ['n','nn'])
E = pd.Index(['35011','35011_F'], name = 'n')

Some of the sectors may not use the energy inputs, or only use either the foreign/domestic types:

In [4]:
m_onlyEdom = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), 
                                   ('KELBM', 'KELB'), 
                                   ('KELB', 'iB'), ('KELB','KEL'), 
                                   ('KEL','L'), ('KEL','KE'), 
                                   ('KE','iM'), ('KE','35011_input')], names = ['n','nn'])
m_onlyEfor = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), 
                                   ('KELBM', 'KELB'), 
                                   ('KELB', 'iB'), ('KELB','KEL'), 
                                   ('KEL','L'), ('KEL','KE'), 
                                   ('KE','iM'), ('KE','35011_F')], names = ['n','nn'])
m_noE = pd.MultiIndex.from_tuples([('KELBM', 'RxE'), 
                                   ('KELBM', 'KELB'), 
                                   ('KELB', 'iB'), ('KELB','KEL'), 
                                   ('KEL','L'), ('KEL','iM')], names = ['n','nn'])

Look at energy demand with domestic and foreign types in rows:

In [5]:
Edf = adj.rc_pd(db_IO.get('vD'), E).unstack('n')

If the sector is not in ```Edf```, the sector does not demand any energy inputs. If the foreign electricity good is NaN, the sector only uses domestic energy. If the domestic sector is NaN, they only use foreign:

In [6]:
s_noE = adj.rc_pd(db_IO.get('s_p'), ('not',Edf))
s_onlyFor = adj.rc_pd(db_IO.get('s_p'), Edf[Edf['35011'].isna()])
s_onlyDom = adj.rc_pd(db_IO.get('s_p'), Edf[Edf['35011_F'].isna()])
s_fullE = adj.rc_pd(db_IO.get('s_p'), ('not', ('or', [s_noE, s_onlyFor, s_onlyDom])))

Establish mappings:

In [7]:
m = pd.MultiIndex.from_tuples(np.hstack([pyDatabases.cartesianProductIndex([s_noE, m_noE]).values,
                                         pyDatabases.cartesianProductIndex([s_onlyFor, m_onlyEfor]).values,
                                         pyDatabases.cartesianProductIndex([s_onlyDom, m_onlyEdom]).values,
                                         pyDatabases.cartesianProductIndex([s_fullE, mFull]).values]),
                              names = ['s','n','nn'])

Replace upper-most level with name of sector:

In [8]:
df = m.to_frame(index=False) 
df.loc[df.n == 'KELBM','n'] = df.loc[df.n == 'KELBM', 's']
m = pd.MultiIndex.from_frame(df)

**Materials nest:**

There are quite a lot of sparsity in the materials nest - so we deal explicitly with it.

In [9]:
E = pd.Index(['35011','35011_F'], name = 'n')

For [s,n] in ```dImport[s,n,nn]```, we define a mapping from ```RxE```to an intermediate good ```RxEym_x``` that is again an aggregate of the domestic/foreign goods ```x,x_F```. We do this for any sectors except the energy goods:

In [10]:
E = pd.Index(['35011','35011_F'], name = 'n') # treat these types of good in a different way (not in materials nest)
df = adj.rc_pd(db_IO.get('dImport'), ('and', [db_IO.get('s_p'), ('not',E)]) # imports for all domestic production sectors, not E
              ).to_frame(index=False).assign(RxEym= lambda x: 'RxEym_'+x['n'], RxE = 'RxE', n = lambda x: x['n']+'_input') # add intermediate goods
m = m.union(pd.MultiIndex.from_frame(df[['s','RxE','RxEym']]).rename(['s','n','nn']))
m = m.union(pd.MultiIndex.from_frame(df[['s','RxEym','n']]).rename(['s','n','nn']))
m = m.union(pd.MultiIndex.from_frame(df[['s','RxEym','nn']]).rename(['s','n','nn']))

For [s,n] in ```dImport_dom[s,n]```, we define a mapping from ```RxE```to the domestic goods:

In [11]:
df = adj.rc_pd(db_IO.get('dImport_dom'), ('and', [db_IO.get('s_p'), ('not',E)])
              ).to_frame(index=False).assign(RxE = 'RxE', n = lambda x: x['n']+'_input')
m = m.union(pd.MultiIndex.from_frame(df[['s','RxE','n']]).rename(['s','n','nn']))

For [s,n] in ```dImport_for[s,n]```, we define a mapping from ```RxE```to the forein goods:

In [12]:
df = adj.rc_pd(db_IO.get('dImport_for'), ('and', [db_IO.get('s_p'), ('not',E)])
              ).to_frame(index=False).assign(RxE = 'RxE')
m = m.union(pd.MultiIndex.from_frame(df[['s','RxE','n']]).rename(['s','n','nn']))

Initialize nesting tree with the specific structure:

In [14]:
tree = nestingTree.tree(name = 'CESnest', tree = m.to_list()) # individual tree
Tree = nestingTree.aggTree(name = 'stdProduction', trees = {tree.name: tree})(namespace = {n+'_input': n for n in db_IO.get('n')})

Add time index to the IO data:

In [15]:
def addT(symbol, t):
    return adjMultiIndex.bc(symbol, t).reorder_levels(['t']+symbol.index.names if 't' not in symbol.index.names else symbol.index.names)
[db_IO.__setitem__(k, addT(db_IO.get(k), glob.db['t'].vals)) for k in db_IO.getTypes(['variable','scalar_variable'])];

Make a copy for later:

In [16]:
db_IO0 = db_IO.copy() # we are going to adjust data along the way; this keeps a copy of the original data

Add durable prices (for now, to solve static calibration):

In [17]:
db_IO['pD'] = db_IO.get('pD_dur').combine_first(db_IO.get('pD'))
db_IO['p'] = db_IO.get('pD_dur').groupby(['t','n']).mean().combine_first(db_IO.get('p'))

### 2. Initialize module

Initialize production module, without any durables at first:

In [18]:
P = CGE_Production.Production(tree=Tree, glob = glob) # initialize module from nesting tree and global settings
aggregateDB.subset_db(db_IO, Tree.db.get('s')) # goes through all symbols in db_IO and only keep the elements that are in the set 's' from Tree.db
aggregateDB.subset_db(db_IO, Tree.get('n')) # goes through all symbols in db_IO and only keep the elements that are in the set 'n' from Tree.db
robust.robust_merge_dbs(P.s.db, db_IO, priority = 'second') # Merge IO data into the database of the module; if a symbol is in both, prioritize records from the second database

Add value shares:

In [19]:
v = valueShares.valueShares(Tree, db_IO.copy())
v.compile() # set up model structure, and make sure to initialize symbols if they are not yet defined in the database 
v.write(); # write GAMS code used for the model
m = v.run(exportTo = d['work'],ws=ws) # solve the "model".

Use value shares to initialize variables:
* Outputs and inputs are provided by IO data.
* For intermediate goods, assume a price of 1 (default option in the class, so we don't have to do anything) and set value share = quantity.
* Set share parameters to the ones identified by value share system.

In [20]:
gpyDB.GpyDBs_AOM_Second(P.s.db, gpy(adj.rc_pd(m.out_db.get('vD'), P.get('int')).rename('qD'))) # set intermediate goods levels
gpyDB.GpyDBs_AOM_Second(P.s.db, gpy(m.out_db.get('mu').xs(P.get('t0')[0]).rename('mu'))) # set intermediate goods levels

Set tax rate to fit revenue collected in baseline year:

In [21]:
# P.s.db['tauLump'] = db_IO.get('TotalTax').rename('tauLump')
P.s.db['tauS'] = adjMultiIndex.applyMult(db_IO.get('TotalTax'), P.get('output')) / P.get('qS')

### 3. Static calibration

In [22]:
P.compile(initDB=True) # set up model structure, and make sure to initialize symbols if they are not yet defined in the database (initDB = True)
P.s.setstate('C') # set to calibration state
P.write(); # write GAMS code
mStatic = P.run(exportTo = d['work'],ws=ws,**{'cns': 'CONOPT4'}) # solve the model using CONOPT4.

In [26]:
mStatic.out_db.get('mu')

s      n            nn     
0600a  0600a        KELB       1.709659
                    RxE        0.144326
       E            35011      0.864481
                    35011_F    0.135519
       KE           E          0.001613
                                 ...   
       RxEym_55560  55560_F    0.089579
       RxEym_64000  64000      0.966897
                    64000_F    0.033103
       RxEym_71000  71000      0.842675
                    71000_F    0.157325
Name: level, Length: 86, dtype: object